In [1]:
import numpy as np
from tkinter import *

In [2]:
#This function takes list containing Cumulative probability distribution as argument _
#and  generates random sample from Cumulative probability distribution
def random_sample_CPD(cpd):
    cp=np.array(cpd)
    p=np.random.rand(1)#generate random probabity from uniform distribution 
    a=np.where(p<=cp)
    return a[0][0]+1

In [3]:
# this is the main function which implements the game, it take number of moves as argument
def game(moves):
    cdp_dice=[0.167,0.33,0.5,0.67,0.83,1.0] #Cumulative prob. distribution of rolling single dice
    cpd_chance=[0.11,1] # I am not considering selling and buying chance card, 
                        # only 1 card for goto jail and 8 cards for go to somewhere so for chance card probabibity_
                        #would be 1/9 and 8/9, so cumulative prob. distrib. is [0.11,1]

    prob=[]
    cpd_chance_move=[0.125,0.25,0.375,0.5,0.625,0.75,0.875,1] #Cumulative probabbility distribution of 8 move somewhere chance cards
    counter=0
    cpd_community_chest=[0.5,1]   #Cumulative prob. distribution of cummunity chest cards
    counter=0
    step=0
    doubles=0
    
    board=["dontcare","go","mediterranean","communitychest","dontcare","dontcare","readingrail","dontcare","chance","dontcare","dontcare",
           "jail","stcharles","electrict","dontcare","dontcare","pennsylveniarail","dontcare","communitychest","dontcare","dontcare",
           "dontcare","dontcare","chance","dontcare","illinoisavenue","borailroad","dontcare","dontcare","water","dontcare","gotojail",
           "dontcare","dontcare","communitychest","dontcare","shortline","chance","dontcare","dontcare","boardwalk"]
    squars={"mediterranean":0,"go":0,"boardwalk":0,"readingrail":0,"pennsylveniarail":0,"borailroad":0,"shortline":0}
    
    for i in range(moves):
    
        while True:
            # rolling of dice i.e random sample generation from CPD of rolling the dice
            dice1=random_sample_CPD(cdp_dice)
            dice2=random_sample_CPD(cdp_dice)
            
            if (counter+dice1+dice2)%40 == 0:
                counter=40
            else:
                counter=(counter+dice1+dice2)%40
            #If after dice rolls counter comes at one of the given squar , program handles them accordingly
            if board[counter] == "mediterranean":
                squars["mediterranean"]=squars["mediterranean"]+1
            elif board[counter] == "go":
                squars["go"]=squars["go"]+1
            elif board[counter] == "boardwalk":
                squars["boardwalk"]=squars["boardwalk"]+1
            elif board[counter] == "readingrail":
                squars["readingrail"]=squars["readingrail"]+1
            elif board[counter] == "pennsylveniarail":
                squars["pennsylveniarail"]=squars["pennsylveniarail"]+1
            elif board[counter] == "borailroad":
                squars["borailroad"]=squars["borailroad"]+1
            elif board[counter] == "shortline":
                squars["shortline"]=squars["shortline"]+1
            elif board[counter] == "gotojail":
                counter=11  #Go to jail
            elif board[counter] == "chance":  #implementation of chance card
                num=random_sample_CPD(cpd_chance) # generate random sample from CPD of chance card
                #if num==1 means goto jail selected if num ==2 means one of the 8 move card selected
                if num == 1:
                    counter=11  # Go to jail
                if num == 2: #Move somewhere card selected
                    moveto=random_sample_CPD(cpd_chance_move) # Generate random sample from CPD of move somewhere card
                    if moveto == 1:  # move to reading rail road
                        counter= board.index("readingrail")
                        squars["readingrail"]=squars["readingrail"]+1
                    elif moveto == 2: # move to broadwalk
                        counter= board.index("boardwalk")
                        squars["boardwalk"]=squars["boardwalk"]+1
                    elif moveto == 3: # move to illinois avenue
                         counter= board.index("illinoisavenue") 
                    elif moveto == 4:# move to st charles
                         counter= board.index("stcharles") 
                    elif moveto == 5: # move to Go square
                         counter= board.index("go")
                         squars["go"]=squars["go"]+1
                    elif moveto == 6: # move to nearest railway station
                        n1= np.abs(counter-board.index("readingrail"))
                        n2= np.abs(counter-board.index("pennsylveniarail"))
                        n3= np.abs(counter-board.index("borailroad"))
                        if min(n1,n2,n3) == n1:
                            counter=board.index("readingrail")
                            squars["readingrail"]=squars["readingrail"]+1
                        elif min(n1,n2,n3) == n2:
                            counter=board.index("pennsylveniarail")
                            squars["pennsylveniarail"]=squars["pennsylveniarail"]+1
                        elif min(n1,n2,n3) == n3:
                            counter=board.index("borailroad")
                            squars["borailroad"]=squars["borailroad"]+1

                    elif moveto == 7: # move to nearest utility
                        n1= np.abs(counter-board.index("electrict"))
                        n2= np.abs(counter-board.index("water"))
                        if min(n1,n2) == n1:
                            counter=board.index("electrict")
                        elif min(n1,n2) == n2:
                            counter=board.index("water")
                    elif moveto == 8: #move to community chest square
                        counter=counter-3
                        if board[counter]== "communitychest":
                            num=random_sample_CPD(cpd_community_chest)
                            if num==1:
                                counter=11
                            elif num == 2:
                                counter= board.index("go")
                                squars["go"]=squars["go"]+1



            elif board[counter] == "communitychest":
                num=random_sample_CPD(cpd_community_chest)
                if num==1:
                    counter=11
                elif num == 2:
                    counter= board.index("go")
                    squars["go"]=squars["go"]+1


            #Each move runs under a ininit while loop , which breaks only when dice1 != dice2 or doubles occured three times
            #if doubles occures three times then go to jain
            if dice1 != dice2:
                doubles=0
                break
            else:
                doubles+=1
                if doubles == 3:
                    doubles=0
                    counter=11 # go to jail on cotinuous 3 doubles
                    break

    #Calculating cumulative probability of Squares
    for i in squars:
        prob.append(squars[i]/moves)
       
    return np.array(prob)


In [19]:
#code for running simulation 1000 times, having 100 moves in each game
numofgame=1000
n=np.zeros((7))
for i in range(numofgame):
    p=game(100)
    n=n+p
f=n/1000
print(np.round(f,3))

[0.019 0.078 0.032 0.041 0.037 0.037 0.029]


In [12]:
tk=Tk()
tk.title("Monopoly")
tk.resizable(0,0) #disable resizing of window
tk.wm_attributes("-topmost",1)  # Keeps the window at top of all windows
canvas=Canvas(tk,width=700,height=720,bd=0,highlightthickness=0) # creates a canvas of 700x720

#creates board rectangle
canvas.create_rectangle(20, 20, 680, 680, fill="pale goldenrod")
canvas.create_text((350, 350), text="MONOPOLY",fill="black",font=("Times", 30))

# Creates all the squares
canvas.create_rectangle(20, 20, 100, 100, fill="red")
canvas.create_text((60, 48), text="Free",fill="white",font=("Times", 14))
canvas.create_text((60, 62), text="Parking",fill="white",font=("Times", 14))

canvas.create_rectangle(600, 20, 680, 100, fill="blue")
canvas.create_text((640, 60), text="GO to Jail",fill="white",font=("Times", 13))

canvas.create_rectangle(20, 600, 100, 680, fill="light goldenrod")
canvas.create_rectangle(40, 600, 100, 660, fill="red")
jail=canvas.create_text((70, 630), text="Jail",fill="white",font=("Times", 14))

canvas.create_rectangle(600, 600, 680, 680, fill="red")

for i in range(0,9):
    canvas.create_rectangle(100+i*55.55, 20, 155.55+i*55.55, 100, fill="light goldenrod")
for i in range(0,9):
    canvas.create_rectangle(20,100+i*55.55, 100, 155.55+i*55.55, fill="light goldenrod")
for i in range(0,9):
    canvas.create_rectangle(100+i*55.55,600, 155.55+i*55.55, 680, fill="light goldenrod")
for i in range(0,9):
    canvas.create_rectangle(600,100+i*55.55,680,155.55+i*55.55, fill="light goldenrod")

# writes text withing squares     
canvas.create_text((640, 640), text="GO",fill="white",font=("Times", 15))
go_prob=canvas.create_text((640, 658), text="",fill="white",font=("Times", 13))

canvas.create_text((640, 565), text="Broadwalk",font=("Times", 9))
brd_prob=canvas.create_text((640, 585), text="",font=("Times", 13))

canvas.create_text((572, 630), text="Mediteran",font=("Times", 9))
med_prob=canvas.create_text((572, 658), text="",font=("Times", 13))

canvas.create_text((350, 630), text="Reading",font=("Times", 9))
reading_prob=canvas.create_text((350, 658), text="",font=("Times", 13))

canvas.create_text((350, 50), text="Bo Rail",font=("Times", 9))
Bo_prob=canvas.create_text((350, 70), text="",font=("Times", 13))

canvas.create_text((65, 340), text="Pen Rail",font=("Times", 9))
Pen_prob=canvas.create_text((65, 360), text="",font=("Times", 13))

canvas.create_line(608, 620,620,610, fill="white", width=5)
canvas.create_line(606, 620,670,620, fill="white", width=5)
canvas.create_line(608, 620,620,630, fill="white", width=5)

#buttons for running and clearing 
runall = Button(canvas, text = "Run All", command =run_complete , anchor = W)
runall.configure(width = 7, activebackground = "green", relief = FLAT,bg = "gold")
canvas.create_window(480, 690, anchor=NW, window=runall)

runsingle = Button(canvas, text = "Run Single", command =run_single , anchor = W)
runsingle.configure(width = 8, activebackground = "green", relief = FLAT,bg = "gold")
canvas.create_window(547, 690, anchor=NW, window=runsingle)


clear = Button(canvas, text = "Clear", command =clear_board , anchor = W)
clear.configure(width = 7, activebackground = "green", relief = FLAT,bg = "gold")
canvas.create_window(620, 690, anchor=NW, window=clear)

#creates text boxes for taking input
canvas.create_text((70, 700), text="Number of games",font=("Times", 11))
e1 = Entry(canvas)
e1.configure(bg="yellow")
canvas.create_window(170,700,width=60,window = e1)

canvas.create_text((270, 700), text="Moves per game",font=("Times", 11))
e2 = Entry(canvas)
e2.configure(bg="yellow")
canvas.create_window(360,700,width=60,window = e2)

canvas.pack()
tk.update()
tk.mainloop()



In [7]:
# this function is called when user clickes on run all button
 numofgame=int(e1.get())
    numofmoves=int(e2.get())
def run_complete():
   
    n=np.zeros((7))
    for i in range(1000):
        p=game(100)
        n=n+p

    f_p=np.round(n/numofgame,3)
    canvas.itemconfig(go_prob, text=f_p[1])
    canvas.itemconfig(brd_prob, text=f_p[2])
    canvas.itemconfig(med_prob, text=f_p[0])
    canvas.itemconfig(reading_prob, text=f_p[3])
    canvas.itemconfig(Bo_prob, text=f_p[5])
    canvas.itemconfig(Pen_prob, text=f_p[4])
    


In [8]:
#this function is called to clear all the probabilities
def clear_board():
    
    canvas.itemconfig(go_prob, text="")
    canvas.itemconfig(brd_prob, text="")
    canvas.itemconfig(med_prob, text="")
    canvas.itemconfig(reading_prob, text="")
    canvas.itemconfig(Bo_prob, text="")
    canvas.itemconfig(Pen_prob, text="")
    

In [13]:
#this function is called to run a game of 100 moveis
def run_single():
    p=game(100)
    f_p=np.round(p,3)
    canvas.itemconfig(go_prob, text=f_p[1])
    canvas.itemconfig(brd_prob, text=f_p[2])
    canvas.itemconfig(med_prob, text=f_p[0])
    canvas.itemconfig(reading_prob, text=f_p[3])
    canvas.itemconfig(Bo_prob, text=f_p[5])
    canvas.itemconfig(Pen_prob, text=f_p[4])


In [15]:
tk=Tk()
tk.title("Monopoly")
tk.resizable(0,0) #disable resizing of window
tk.wm_attributes("-topmost",1)  # Keeps the window at top of all windows
canvas=Canvas(tk,width=700,height=720,bd=0,highlightthickness=0) # creates a canvas of 700x720

#creates board rectangle
canvas.create_rectangle(20, 20, 680, 680, fill="pale goldenrod")
canvas.create_text((350, 350), text="MONOPOLY",fill="black",font=("Times", 30))

# Creates all the squares
canvas.create_rectangle(20, 20, 100, 100, fill="red")
canvas.create_text((60, 48), text="Free",fill="white",font=("Times", 14))
canvas.create_text((60, 62), text="Parking",fill="white",font=("Times", 14))

canvas.create_rectangle(600, 20, 680, 100, fill="blue")
canvas.create_text((640, 60), text="GO to Jail",fill="white",font=("Times", 13))

canvas.create_rectangle(20, 600, 100, 680, fill="light goldenrod")
canvas.create_rectangle(40, 600, 100, 660, fill="red")
jail=canvas.create_text((70, 630), text="Jail",fill="white",font=("Times", 14))

canvas.create_rectangle(600, 600, 680, 680, fill="red")

for i in range(0,9):
    canvas.create_rectangle(100+i*55.55, 20, 155.55+i*55.55, 100, fill="light goldenrod")
for i in range(0,9):
    canvas.create_rectangle(20,100+i*55.55, 100, 155.55+i*55.55, fill="light goldenrod")
for i in range(0,9):
    canvas.create_rectangle(100+i*55.55,600, 155.55+i*55.55, 680, fill="light goldenrod")
for i in range(0,9):
    canvas.create_rectangle(600,100+i*55.55,680,155.55+i*55.55, fill="light goldenrod")

# writes text withing squares     
canvas.create_text((640, 640), text="GO",fill="white",font=("Times", 15))
go_prob=canvas.create_text((640, 658), text="",fill="white",font=("Times", 13))

canvas.create_text((640, 565), text="Broadwalk",font=("Times", 9))
brd_prob=canvas.create_text((640, 585), text="",font=("Times", 13))

canvas.create_text((572, 630), text="Mediteran",font=("Times", 9))
med_prob=canvas.create_text((572, 658), text="",font=("Times", 13))

canvas.create_text((350, 630), text="Reading",font=("Times", 9))
reading_prob=canvas.create_text((350, 658), text="",font=("Times", 13))

canvas.create_text((350, 50), text="Bo Rail",font=("Times", 9))
Bo_prob=canvas.create_text((350, 70), text="",font=("Times", 13))

canvas.create_text((65, 340), text="Pen Rail",font=("Times", 9))
Pen_prob=canvas.create_text((65, 360), text="",font=("Times", 13))

canvas.create_line(608, 620,620,610, fill="white", width=5)
canvas.create_line(606, 620,670,620, fill="white", width=5)
canvas.create_line(608, 620,620,630, fill="white", width=5)

n=np.zeros((7))
for i in range(1000):
    p=game(100)
    n=n+p

f_p=np.round(n/numofgame,3)
canvas.itemconfig(go_prob, text=f_p[1])
canvas.itemconfig(brd_prob, text=f_p[2])
canvas.itemconfig(med_prob, text=f_p[0])
canvas.itemconfig(reading_prob, text=f_p[3])
canvas.itemconfig(Bo_prob, text=f_p[5])
canvas.itemconfig(Pen_prob, text=f_p[4])


canvas.pack()
tk.update()
tk.mainloop()


